In [6]:
# Simple Multiagent System with Real LLMs
import openai
import time

OPENAI_API_KEY = ""  # Replace with actual key
client = openai.OpenAI(api_key=OPENAI_API_KEY)


# BASE AGENT CLASS
class Agent:
    def __init__(self, name, role, model="gpt-3.5-turbo"):
        self.name = name
        self.role = role
        self.model = model
        self.memory = []

    def call_llm(self, prompt):

        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": f"You are {self.name}, a {self.role} agent."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300,
                temperature=0.7
            )
            result = response.choices[0].message.content
            print(f"{self.name}: {result[:60]}...")
            return result
        except Exception as e:
            print(f" API failed for {self.name}: {e}")
            return f"Mock response from {self.name}: {prompt[:50]}..."

    def process(self, task):

        prompt = f"As a {self.role}, {task}"
        result = self.call_llm(prompt)
        self.memory.append({"task": task, "result": result})
        return result

    def send_to(self, other_agent, message):

        print(f" {self.name} → {other_agent.name}: {message[:40]}...")
        return other_agent.process(f"Process this from {self.name}: {message}")

# SPECIALIZED AGENTS

class Coordinator(Agent):
    def __init__(self):
        super().__init__("Boss", "project coordinator", "gpt-4")
        self.team = []

    def add_agent(self, agent):
        self.team.append(agent)
        print(f"➕ Added {agent.name} to team")

    def delegate_project(self, project):
        print(f"\n COORDINATING PROJECT: {project}")
        plan = self.process(f"Create a plan for: {project}")

        results = {}
        for agent in self.team:
            task = f"Work on project '{project}' using your {agent.role} skills"
            results[agent.name] = agent.process(task)

        return {"plan": plan, "results": results}

class Researcher(Agent):
    def __init__(self):
        super().__init__("Scout", "researcher", "gpt-3.5-turbo")

    def research(self, topic):
        return self.process(f"research and gather information about: {topic}")

class Analyst(Agent):
    def __init__(self):
        super().__init__("Brain", "data analyst", "gpt-4")

    def analyze(self, data):
        return self.process(f"analyze this data and provide insights: {data}")

class Writer(Agent):
    def __init__(self):
        super().__init__("Pen", "content writer", "gpt-3.5-turbo")

    def write_report(self, content):
        return self.process(f"write a professional report based on: {content}")


# MULTIAGENT SYSTEM

class MultiAgentTeam:
    def __init__(self):
        # Create team
        self.coordinator = Coordinator()
        self.researcher = Researcher()
        self.analyst = Analyst()
        self.writer = Writer()

        # Setup team
        self.coordinator.add_agent(self.researcher)
        self.coordinator.add_agent(self.analyst)
        self.coordinator.add_agent(self.writer)

        print(" Multiagent team created!")

    def execute_project(self, project_description):
        print(f"\n EXECUTING PROJECT")
        print("=" * 40)

        # Step 1: Coordinate
        coordination = self.coordinator.delegate_project(project_description)

        # Step 2: Research
        research_data = self.researcher.research(project_description)

        # Step 3: Analyze research
        analysis = self.researcher.send_to(self.analyst, research_data)

        # Step 4: Write report
        final_report = self.analyst.send_to(self.writer, analysis)

        print(f"\n PROJECT COMPLETED!")
        return {
            "coordination": coordination,
            "research": research_data,
            "analysis": analysis,
            "report": final_report
        }

    def show_team_status(self):
        print(f"\n TEAM STATUS")
        agents = [self.coordinator, self.researcher, self.analyst, self.writer]
        for agent in agents:
            print(f"  {agent.name} ({agent.role}): {len(agent.memory)} tasks completed")


# DEMO FUNCTIONS

def demo_basic_multiagent():
    print(" MULTIAGENT SYSTEM DEMO")
    print("=" * 40)

    # Create team
    team = MultiAgentTeam()

    # Execute project
    project = "Analyze AI market trends and create business strategy"
    results = team.execute_project(project)

    # Show status
    team.show_team_status()

    return results

def demo_agent_communication():
    print(f"\n AGENT COMMUNICATION")
    print("-" * 30)

    researcher = Researcher()
    analyst = Analyst()

    # Research → Analysis chain
    research = researcher.research("artificial intelligence trends")
    analysis = researcher.send_to(analyst, research)

    print(" Communication chain completed!")

def demo_different_models():

    print(f"\n DIFFERENT LLM MODELS")
    print("-" * 30)

    fast_agent = Agent("Fast", "assistant", "gpt-3.5-turbo")
    smart_agent = Agent("Smart", "analyst", "gpt-4")

    task = "Evaluate autonomous vehicles"

    fast_result = fast_agent.process(task)
    smart_result = smart_agent.process(task)

    print(" Different models demonstrated!")

# MAIN EXECUTION

def run_multiagent_demo():
    print("MULTIAGENT SYSTEM WITH REAL LLMs")
    print("=" * 50)
    print("  Set your OpenAI API key at the top!")

    try:
        # Main demo
        results = demo_basic_multiagent()

        # Additional demos
        demo_agent_communication()
        demo_different_models()

    except Exception as e:
        print(f" Demo failed: {e}")
        print(" Check your API key!")

if __name__ == "__main__":
    run_multiagent_demo()

MULTIAGENT SYSTEM WITH REAL LLMs
  Set your OpenAI API key at the top!
 MULTIAGENT SYSTEM DEMO
➕ Added Scout to team
➕ Added Brain to team
➕ Added Pen to team
 Multiagent team created!

 EXECUTING PROJECT

 COORDINATING PROJECT: Analyze AI market trends and create business strategy
Boss: Phase 1: Market Research

1.1 Identify Key Research Areas: I...
Scout: As a researcher, I would approach the project 'Analyze AI ma...
Brain: Project: Analyze AI market trends and create business strate...
Pen: Title: Analyzing AI Market Trends and Crafting a Successful ...
Scout: Analyzing AI market trends and creating a business strategy ...
 Scout → Brain: Analyzing AI market trends and creating ...
Brain: Based on the information provided by Scout, the key points t...
 Brain → Pen: Based on the information provided by Sco...
Pen: Here is a summary based on the key points provided by Scout ...

 PROJECT COMPLETED!

 TEAM STATUS
  Boss (project coordinator): 1 tasks completed
  Scout (researcher): 2 